In [1]:
# Import useful packeges:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import flatten
import cv2
import os
import pickle
import glob
from matplotlib.pyplot import figure, show, cm, imshow
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.regularizers import l2
from sklearn.preprocessing import LabelBinarizer


Using TensorFlow backend.


I left this code in below, for expanding the image set:
1. Get the base images from http://wwww-ablab.ucsd.edu/~xed/ateam/tl_imgs.tgz
2. Extract the images to directory such as tl_imgs/R, tl_imgs/Y, tl_imgs/G
3. Write images to other places

In [2]:
"""
import os
import random
import glob
import cv2
#base_red_images = glob.glob('tl_imgs/R/*')
#base_yellow_images = glob.glob('tl_imgs/Y/*')
#base_green_images = glob.glob('tl_imgs/G/*')

def write_crop(ofn,i,x,y,dirname):
    '''Writes to filename (n) a 200x300 image (i) centered at the supplied coordinates (x,y).'''
    cropi= i.copy()
    cropi= cropi[(y-225):(y+225),(x-150):(x+150)]
    cropi= cv2.resize(cropi,(200,300),interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(os.path.join(dirname, ofn),cropi)
    
def write_images(images, dirname):
    for img in images:
        c=0
        print("Processing: %s"%img)
        i= cv2.imread(img)
        x,y= 400,300
        name = img[-10:]
        write_crop(name[:-4]+('_crop%02d.png'%c),i,x,y,dirname)
        for yoffset in range(-40,60,20):
            for xoffset in range(-200,225,25):
                r= random.randint(-20,20)
                write_crop(name[:-4]+('_crop%02d.png'%c),i,x+xoffset+r,y+yoffset+r,dirname)
                c+=1
            
#write_images(base_green_images,'tl_imgs/G_train')
#write_images(base_red_images,'tl_imgs/R_train')
#write_images(base_yellow_images,'tl_imgs/Y_train')
"""

'\nimport os\nimport random\nimport glob\nimport cv2\n#base_red_images = glob.glob(\'tl_imgs/R/*\')\n#base_yellow_images = glob.glob(\'tl_imgs/Y/*\')\n#base_green_images = glob.glob(\'tl_imgs/G/*\')\n\ndef write_crop(ofn,i,x,y,dirname):\n    \'\'\'Writes to filename (n) a 200x300 image (i) centered at the supplied coordinates (x,y).\'\'\'\n    cropi= i.copy()\n    cropi= cropi[(y-225):(y+225),(x-150):(x+150)]\n    cropi= cv2.resize(cropi,(200,300),interpolation = cv2.INTER_CUBIC)\n    cv2.imwrite(os.path.join(dirname, ofn),cropi)\n    \ndef write_images(images, dirname):\n    for img in images:\n        c=0\n        print("Processing: %s"%img)\n        i= cv2.imread(img)\n        x,y= 400,300\n        name = img[-10:]\n        write_crop(name[:-4]+(\'_crop%02d.png\'%c),i,x,y,dirname)\n        for yoffset in range(-40,60,20):\n            for xoffset in range(-200,225,25):\n                r= random.randint(-20,20)\n                write_crop(name[:-4]+(\'_crop%02d.png\'%c),i,x+xoffset+

In [3]:
def normalize_image(image):
    r, g, b = cv2.split(image)
    r = (r - 128)/128
    g = (g - 128)/128
    b = (b - 128)/128
    return cv2.merge((r, g, b))

# scale images depending on extension/image type
def load_image(image_path):

    image = cv2.imread(image_path) 
    image = cv2.resize(image,(32,32), interpolation = cv2.INTER_CUBIC)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    #image = normalize_image(image)

    '''
    # scale image (0-255)
    if image_path[-4:] == '.png':
        image = image.astype(np.float32)*255
    '''

    # remove alpha channel if present
    if image.shape[2] == 4:
        b, g, r, a = cv2.split(image)
        image = np.dstack((r,g,b))

    return image

Then set up the images - basically this makes training images, and testing images

In [4]:
# I changed these directories from R_train to R, etc so I did use the expanded set
tr_red_images = glob.glob('tl_imgs/R/*')
tr_yellow_images = glob.glob('tl_imgs/Y/*')
tr_green_images = glob.glob('tl_imgs/G/*')

# print out how many images there were in the data
print('# red images: ', len(tr_red_images))
print('# yellow images: ', len(tr_yellow_images))
print('# green images: ', len(tr_green_images))

images = []
labels = []

# Set how many images you want to use on training and testing
# should replace this DataGenerator because this is clumsy and I am neglecting lots of data
# This is for each colour
train_num = 5000
test_num = 5000

#randomly select images from red set and remove them so that they cannot be selected again
for i in range(train_num):
    choice = random.randint(0, len(tr_red_images) - 1)
    image = load_image(tr_red_images[choice])
    images.append(image)
    labels.append(2)
    del tr_red_images[choice]
    
# yellow
for i in range(train_num):
    choice = random.randint(0, len(tr_yellow_images) - 1)
    image = load_image(tr_yellow_images[choice])
    images.append(image)
    labels.append(1)
    del tr_yellow_images[choice]
    
# green
for i in range(train_num):
    choice = random.randint(0, len(tr_green_images) - 1)
    image = load_image(tr_green_images[choice])
    images.append(image)
    labels.append(0)
    del tr_green_images[choice]
    
X_train = np.array(images)
y_train = np.array(labels)

print('Shape of X_train: ', X_train.shape)
print('Shape of y_train: ', y_train.shape)
   
# Now continue to work on the test set, ignoring images that were previously added    
    
images = []
labels = []

# Test set    
# red
for i in range(test_num):
    choice = random.randint(0, len(tr_red_images) - 1)
    image = load_image(tr_red_images[choice])
    images.append(image)
    labels.append(2)
    del tr_red_images[choice]
    
# yellow
for i in range(test_num):
    choice = random.randint(0, len(tr_yellow_images) - 1)
    image = load_image(tr_yellow_images[choice])
    images.append(image)
    labels.append(1)
    del tr_yellow_images[choice]
    
# green
for i in range(test_num):
    choice = random.randint(0, len(tr_green_images) - 1)
    image = load_image(tr_green_images[choice])
    images.append(image)
    labels.append(0)
    del tr_green_images[choice]
    
X_test = np.array(images)
y_test = np.array(labels)

print('Shape of X_test: ', X_test.shape)
print('Shape of y_test: ', y_test.shape)

# red images:  48535
# yellow images:  36805
# green images:  50320
Shape of X_train:  (15000, 32, 32, 3)
Shape of y_train:  (15000,)
Shape of X_test:  (15000, 32, 32, 3)
Shape of y_test:  (15000,)


In [5]:
X_train, y_train = shuffle(X_train, y_train)
#X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=1024)
X_test, y_test = shuffle(X_test, y_test)
print('Done with shuffle and split')

Done with shuffle and split


In [6]:
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_one_hot, epochs=30, validation_split=0.2, verbose=2)



Train on 12000 samples, validate on 3000 samples
Epoch 1/30
8s - loss: 3.2702 - acc: 0.6625 - val_loss: 0.4639 - val_acc: 0.7512
Epoch 2/30
4s - loss: 0.2946 - acc: 0.8502 - val_loss: 0.1467 - val_acc: 0.9591
Epoch 3/30
4s - loss: 0.1370 - acc: 0.9607 - val_loss: 0.1173 - val_acc: 0.9662
Epoch 4/30
4s - loss: 0.1169 - acc: 0.9683 - val_loss: 0.1075 - val_acc: 0.9700
Epoch 5/30
4s - loss: 0.1086 - acc: 0.9707 - val_loss: 0.1064 - val_acc: 0.9689
Epoch 6/30
4s - loss: 0.1048 - acc: 0.9719 - val_loss: 0.0816 - val_acc: 0.9700
Epoch 7/30
3s - loss: 0.0884 - acc: 0.9651 - val_loss: 0.0540 - val_acc: 0.9707
Epoch 8/30
4s - loss: 0.0498 - acc: 0.9748 - val_loss: 0.0481 - val_acc: 0.9758
Epoch 9/30
4s - loss: 0.0437 - acc: 0.9778 - val_loss: 0.0505 - val_acc: 0.9749
Epoch 10/30
4s - loss: 0.0432 - acc: 0.9776 - val_loss: 0.0468 - val_acc: 0.9769
Epoch 11/30
4s - loss: 0.0419 - acc: 0.9794 - val_loss: 0.0423 - val_acc: 0.9789
Epoch 12/30
4s - loss: 0.0413 - acc: 0.9792 - val_loss: 0.0454 - val_

In [7]:
# Evaluate
y_one_hot_test = label_binarizer.fit_transform(y_test)
metrics = model.evaluate(X_test, y_one_hot_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

from keras.models import load_model
model.save('./keras_model.h5')

14944/15000 [============================>.] - ETA: 0sloss: 0.047898234704985894
acc: 0.9818666851043701


In [8]:
# check the length of the remaining images for just to be sure that we are in fact removing images
print('# red images: ', len(tr_red_images))
print('# yellow images: ', len(tr_yellow_images))
print('# green images: ', len(tr_green_images))

# red images:  38535
# yellow images:  26805
# green images:  40320


In [9]:
# Load the model and check the probabilities
# pick a red, yellow, green
r_img = load_image(tr_red_images[0])
y_img = load_image(tr_yellow_images[0])
g_img = load_image(tr_green_images[0])

In [10]:
r_img = r_img.reshape(1, 32, 32, 3) 
y_prob = model.predict(r_img) 
y_classes = y_prob.argmax(axis=-1)
print(y_classes)

y_img = y_img.reshape(1, 32, 32, 3) 
y_prob = model.predict(y_img) 
y_classes = y_prob.argmax(axis=-1)
print(y_classes)

g_img = g_img.reshape(1, 32, 32, 3) 
y_prob = model.predict(g_img) 
y_classes = y_prob.argmax(axis=-1)
print(y_classes)

[2]
[1]
[0]


Now test on images it has never seen before doing it in the same way as the simulator. Can add other checks if wanted.

In [11]:
choices = {0: "GREEN", 1: "YELLOW", 2: "RED", 3: "UNKNOWN"}

num_images = 0
num_incorrect = 0

#green_length = len(tr_green_images)
#yellow_length = len(tr_yellow_images)
#red_length = len(tr_red_images)

green_length = 5000
yellow_length = 5000
red_length = 5000


for i in range(green_length):
    img = load_image(tr_green_images[i])
    #image = normalize_image(image)
    res = None    
    image = image.reshape(1, 32, 32, 3) 
    classification = model.predict_classes(image)[0]
    result = choices.get(classification, "UNKNOWN")
    num_images += 1
    if result != 'GREEN':
        num_incorrect += 1

success_rate = 0.0
if num_incorrect > 0:
    success_rate = str(100.0-100.0*(float(num_incorrect)/float(num_images)))
if num_incorrect == 0:
    success_rate = 100

print('No Images: ' + str(num_images) + ' incorrect: ' + str(num_incorrect) + ' success rate: ' + str(success_rate) + ' %')

1/1 [==============================] - 0s


1/1 [==============================] - 0s


1/1 [==============================] - 0s


1/1 [==============================] - 0s


1/1 [==============================] - 0s


...

1/1 [==============================] - 0s


1/1 [==============================] - 0s


1/1 [==============================] - 0s


1/1 [==============================] - 0s


1/1 [==============================] - 0s
No Images: 5000 incorrect: 0 success rate: 100 %
